In [1]:
!pip install huggingface_hub

In [2]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer,BertModel,TrainingArguments,Trainer,PreTrainedModel
from transformers import PretrainedConfig
import transformers
from datasets import Dataset
import torch
import torch.nn as nn
from torch.utils.data import RandomSampler, DataLoader,SequentialSampler
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import notebook
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [3]:
from huggingface_hub import notebook_login

In [7]:
notebook_login()

Login successful
Your token has been saved to /Users/adam/.huggingface/token


In [5]:
esgRedditConfig.push_to_hub("ESG-BERT-Reddit")

CommitInfo(commit_url='https://huggingface.co/admation/ESG-BERT-Reddit/commit/4c4926428030c7229f154f29697141c92b1e3b09', commit_message='Upload config', commit_description='', oid='4c4926428030c7229f154f29697141c92b1e3b09', pr_url=None, pr_revision=None, pr_num=None)

In [29]:
class ClassificationModel(nn.Module):
    def __init__(self, bert,num_classes):
        super().__init__()
        self.bert = bert
        self.W = nn.Linear(bert.config.hidden_size, num_classes)
        self.num_classes = num_classes
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        h, _, attn = self.bert(input_ids=input_ids, 
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids).values()
        h_cls = h[:,0,:]
        output = self.W(h_cls)
        return output, attn

In [30]:
finetuned_model = torch.load("esg_bert_reddit.pt")

In [19]:
from ESGBertReddit_model.configuration_ESGBertReddit import ESGRedditConfig
from ESGBertReddit_model.modeling_ESGBertReddit import ClassificationModel,BertModelForESGClassification

In [5]:
ESGRedditConfig.register_for_auto_class("AutoConfig")
ClassificationModel.register_for_auto_class("AutoModel")
BertModelForESGClassification.register_for_auto_class("AutoModelForSequenceClassification")

In [6]:
id2label = {
    0: "D",
    1: "C",
    2: "B",
    3: "A"
}
label2id = {
    "D" : 0,
    "C" : 1,
    "B" : 2,
    "A" : 3
}
esgRedditConfig = ESGRedditConfig(num_classes=4,id2label=id2label,label2id=label2id)
classification = BertModelForESGClassification(esgRedditConfig)

Some weights of the model checkpoint at yiyanghkust/finbert-esg were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
classification.model.bert.load_state_dict(finetuned_model.bert.state_dict())
classification.model.W.load_state_dict(finetuned_model.W.state_dict())

<All keys matched successfully>

In [16]:
classification.model.W.weight == finetuned_model.W.weight

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [20]:
classification.push_to_hub("ESG-BERT-Reddit",commit_message="new model")

CommitInfo(commit_url='https://huggingface.co/admation/ESG-BERT-Reddit/commit/2c19b0f50e74a39840251e2e03d3fa5ea7e0865f', commit_message='new model', commit_description='', oid='2c19b0f50e74a39840251e2e03d3fa5ea7e0865f', pr_url=None, pr_revision=None, pr_num=None)

In [38]:
tokenizer.push_to_hub("ESG-BERT-Reddit")

CommitInfo(commit_url='https://huggingface.co/admation/ESG-BERT-Reddit/commit/ff209c2057b6d5c98c16044d4aa65ac01b03562d', commit_message='Upload tokenizer', commit_description='', oid='ff209c2057b6d5c98c16044d4aa65ac01b03562d', pr_url=None, pr_revision=None, pr_num=None)

In [37]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-esg',model_max_length=256)
model = AutoModelForSequenceClassification.from_pretrained("admation/ESG-BERT-Reddit",trust_remote_code=True)
pred = transformers.pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True,padding=True,truncation=True)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Some weights of the model checkpoint at yiyanghkust/finbert-esg were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The model 'BertModelForESGClassification' is not supported for text-classification.

In [38]:
import shap

In [39]:
explainer = shap.Explainer(pred)
shap_values = explainer(["hello world","environment friendly"])

In [40]:
shap.plots.text(shap_values)

In [11]:
a = MyPipeline(model=classification,tokenizer=tokenizer,padding=True,truncation=True)
a("Hello World")

NameError: name 'MyPipeline' is not defined

In [59]:
classification(*tokenizer(["Hello World"]))

TypeError: forward() takes from 2 to 3 positional arguments but 4 were given

In [8]:
tokenizer = transformers.AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-emotion", use_fast=True)

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]